# 신경망 학습
> **학습**이란, 훈련 데이터로부터 신경망 내의 가중치 매개변수의 최적값을 찾아내는 과정을 의미한다.  
> 이때, **손실 함수**를 최소화함으로써 최적값을 찾는다.

## Loss function
### Mean Squared Error, MSE
평균 제곱 오차 MSE 는 다음과 같다.
$$
\begin{align}
\text{RSS}(w) &\triangleq \sum_{n=1}^{N} (y_{n} - w^{T} x_{n})^{2} \\
\text{MSE}(w) &\triangleq \frac{1}{N}\text{RSS}(w)
\end{align}
$$
즉, 입력-출력 쌍 $(x_n,y_n)$에 대해서, 실제 출력값과 모델 예측값의 차이를 제곱 평균 한 것과 같다.  

#### CODE

In [ ]:
import numpy as np 

def mean_squared_error(y, x):
    return np.mean((y - x) ** 2)

In [ ]:
x = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
y = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])
print(mean_squared_error(y, x))

0.019500000000000007


### Cross Entropy Error, CEE
교차 엔트로피 오차는 다음과 같다.
$$
\begin{align}
\text{CEE}(w) &\triangleq -\sum_{k=1}^{K} y_{k} \log \hat{y}_{k}
\end{align}
$$

이때 $y_k$는 실제 참값, $\hat{y}_k$는 모델이 예측한 값을 의미한다.

#### CODE

In [ ]:
def cross_entropy_error(y_pred, y_true):
    delta = 1e-7
    return -np.sum(y_true * np.log(y_pred + delta)) # add delta to avoid log(0)

In [ ]:
y_true = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
y_pred = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])
print(cross_entropy_error(y_pred, y_true))

0.510825457099338


### 미니배치 학습
이전까지는 데이터 한개에 대한 손실 함수를 보았으므로, 다수의 데이터에 대한 손실 함수를 구현해보자. 위의 교차 엔트로피 오차의 식을 확장시키면, 다음과 같다.
$$
\text{CEE}(w) \triangleq - \frac{1}{N} \sum_{n=1}^{N}\sum_{k=1}^{K} y_{nk}\log(\hat{y}_{nk})
$$
이떄 $y_{nk}$는 $n$번째 데이터의 $k$번째 참값이고, $\hat{y}_{nk}$은 같은 부분의 모델이 예측한 값을 의미한다.  
**미니배치 학습**은 주어진 데이터 전체를 사용하는 것이 아닌, 데이터의 일부만을 사용하여 학습을 진행하는 방법을 말한다.  
이전에 살펴본 MNIST 데이터셋을 사용하여, 미니배치 학습을 구현해보자.

#### CODE

##### 데이터 준비

In [27]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print("x_train.shape =", x_train.shape)  # (60000, 28, 28)
print("y_train.shape =", y_train.shape)  # (60000,)
print("x_test.shape =", x_test.shape)   # (10000, 28, 28)
print("y_test.shape =", y_test.shape)   # (10000,)

x_train.shape = (60000, 28, 28)
y_train.shape = (60000,)
x_test.shape = (10000, 28, 28)
y_test.shape = (10000,)


미니배치를 위해서 주어진 데이터에서 무작위로 N개 추출하는 방법은 `np.random.choice()`를 사용한다.  
다음은 $N=10$일 때의 경우이다.

In [30]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
y_batch = y_train[batch_mask]

In [31]:
print("x_batch.shape =", x_batch.shape)  # (10, 28, 28)
print("y_batch.shape =", y_batch.shape)  # (10,)

x_batch.shape = (10, 28, 28)
y_batch.shape = (10,)


##### 교차 엔트로피 구현

In [64]:
def cross_entropy_error(y_pred, y_true):
    if y_pred.ndim == 1:
        y_pred = y_pred.reshape(1, y_pred.size)
        y_true = y_true.reshape(1, y_true.size)

    batch_size = y_pred.shape[0]
    delta = 1e-7
    #return -np.sum(y_true * np.log(y_pred + delta)) / batch_size
    return -np.sum(np.log(y_pred[np.arange(batch_size), y_true] + delta)) / batch_size

## 경사 하강법
이렇게 구현한 교차 엔트로피 오차, 즉, 손실 함수를 최소화함으로써 모델의 가중치 매개변수를 찾을 수 있다.  
손실 함수의 최솟값은 미분을 통해, 함수의 기울기만큼 가중치 매개변수를 변화시켜가는 과정을 반복하여 찾는다.  
이러한 방법을 **경사 하강법** 이라고 한다.  
수식으론 다음과 같다.  
$$
\begin{align}
x_0 &\triangleq x_0 - \alpha \frac{\partial f}{\partial x_0} \\
x_1 &\triangleq x_1 - \alpha \frac{\partial f}{\partial x_1} \\
&~~~\vdots
\end{align}
$$
이때 $\alpha$를 학습률(Learning rate)라고 한다. 값에 따라 각 매개변수의 값을 얼마나 변화시키는지를 정한다.  
적절한 학습률 값을 정하는 것은 중요한데, 너무 작으면 손실 함수가 최솟값에 가깝게 수렴하는데 속도가 너무 오래 걸리고, 너무 크면 손실 함수가 최솟값에 가깝게 수렴하지 못하고 그 주위에서 '진동' 할 수 있다.

#### CODE

In [33]:
def numerical_gradient(f, x):
    h = 1e-4  # 0.0001
    grad = np.zeros_like(x)

    for idx in range(x.size):
        tmp_val = x[idx]

        # f(x + h)
        x[idx] = tmp_val + h
        fxh1 = f(x)

        # f(x - h)
        x[idx] = tmp_val - h
        fxh2 = f(x)

        grad[idx] = (fxh1 - fxh2) / (2 * h)
        x[idx] = tmp_val  # restore value

    return grad


def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x.copy()

    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad

    return x

In [34]:
def fuction_2(x):
    return x[0]**2 + x[1]**2

init_x = np.array([-3.0, 4.0])
result = gradient_descent(fuction_2, init_x=init_x, lr=0.1, step_num=100)
print("result =", result)  # [ -6.11110793e-10   8.14814391e-10]

result = [-6.11110793e-10  8.14814391e-10]


In [35]:
# 학습률이 큰 경우
init_x = np.array([-3.0, 4.0])
result = gradient_descent(fuction_2, init_x=init_x, lr=10.0, step_num=100)
print("result =", result)  # [ -2.58983747e+13   3.45244996e+13]

result = [-2.58983747e+13 -1.29524862e+12]


In [36]:
# 학습률이 작은 경우
init_x = np.array([-3.0, 4.0])
result = gradient_descent(fuction_2, init_x=init_x, lr=1e-10, step_num=100)
print("result =", result)  # [ -2.99999999   3.99999999]

result = [-2.99999994  3.99999992]


### 신경망에서의 기울기
이를 신경망 학습에 적용하자. 신경망에서 가중치 $\bold{W}$, 손실 함수 $L$에 대해서, 기울기는 다음과 같이 나타낼 수 있다.  
$$
\begin{align}
\bold{W} &= \begin{pmatrix}
w_{11} & w_{12} & w_{13} \\
w_{21} & w_{22} & w_{23}
\end{pmatrix} \\\\
\frac{\partial L}{\partial \bold{W}} &= \begin{pmatrix}
\frac{\partial L}{\partial w_{11}} & \frac{\partial L}{\partial w_{12}} & \frac{\partial L}{\partial w_{13}} \\
\frac{\partial L}{\partial w_{21}} & \frac{\partial L}{\partial w_{22}} & \frac{\partial L}{\partial w_{23}}
\end{pmatrix}
\end{align}
$$

#### CODE

In [70]:
from scipy.special import softmax

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2, 3)  # weight initialization

    def predict(self, x):
        return np.dot(x, self.W)

    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        return loss


In [71]:
net = simpleNet()
print("Initial weight =\n", net.W)  

Initial weight =
 [[-0.32688493  0.66694564 -0.33586083]
 [ 0.84330568 -0.52344912 -0.53677751]]


In [72]:
x = np.array([0.6, 0.9])
p = net.predict(x)
print("Predicted class probabilities =", p)  

Predicted class probabilities = [ 0.56284415 -0.07093683 -0.68461625]


In [73]:
print("Predicted class =", np.argmax(p))  

Predicted class = 0


In [74]:
t = np.array([0, 0, 1])  # true label
loss = net.loss(x, t)
print("Loss value =", loss)

Loss value = 2.4266863149107696


In [75]:
def f(W):
    return net.loss(x, t)

def numerical_gradient(f, x):
    h = 1e-4  # 0.0001
    grad = np.zeros_like(x)

    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x)  # f(x+h)

        x[idx] = tmp_val - h
        fxh2 = f(x)  # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)

        x[idx] = tmp_val  # 값 복원
        it.iternext()

    return grad

dW = numerical_gradient(f, net.W)
print("Gradient of weights =\n", dW)

Gradient of weights =
 [[-0.20980036 -0.07461792  0.28441828]
 [-0.31470054 -0.11192688  0.42662741]]


## 학습 알고리즘 구현
지금까지, 신경망 학습에 대해서 살표보았다. 절차는 다음과 같다.  
1. 미니배치 :  
    훈련 데이터 중 일부를 무작이로 추출한다.
2. 기울기 산출 :  
    미니배치 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다.
3. 매개변수 갱신 :   
    가중치 매개변수를 기울기 값에 따라 갱신한다.
4. 위의 과정을 반복한다.  
   
미니배치는 무작위로 선정되기 때문에 이를 **확률적 경사 하강법**이라 한다.  
이제 손글씨 숫자를 학습하는 신경망을 구현하도록 한다.  
그를 위해 은닉층이 1개인 신경망으로 구성한다.


### 신경망 클래스 구현

#### CODE

In [76]:
class LayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # Weight initialization
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = np.maximum(0, a1)  # ReLU activation
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    # Compute loss
    # x: input data, t: true labels
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    # Compute numerical gradients
    # x: input data, t: true labels
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    

In [77]:
net = LayerNet(input_size=784, hidden_size=100, output_size=10)
print(net.params['W1'].shape)  # (784, 100)
print(net.params['b1'].shape)  # (100,)
print(net.params['W2'].shape)  # (100, 10)
print(net.params['b2'].shape)  # (10,)

(784, 100)
(100,)
(100, 10)
(10,)


### 미니배치 학습 구현

In [ ]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

network = LayerNet(input_size=784, hidden_size=50, output_size=10)

x_train = x_train.reshape(x_train.shape[0], 784)
x_test = x_test.reshape(x_test.shape[0], 784)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

iters_num = 1000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

iter_per_epoch = max(train_size / batch_size, 1)

train_loss_list = []
train_acc_list = []
test_acc_list = []

for i in range(iters_num):
    # Mini-batch selection
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    y_batch = y_train[batch_mask]
    
    # Gradient calculation
    grads = network.numerical_gradient(x_batch, y_batch)
    
    # Parameter update
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grads[key]
    
    # Record loss
    loss = network.loss(x_batch, y_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, y_train)
        test_acc = network.accuracy(x_test, y_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(f"Epoch {i // iter_per_epoch}: Train Accuracy = {train_acc}, Test Accuracy = {test_acc}")
    

Iteration 0, Loss: 6.906243988997322


KeyboardInterrupt: 

여기서 수치 미분법을 사용하였는데, 이는 속도가 매우 느리므로, 코드 실행은 생략한다.